In [1]:

import os
import pandas as pd
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [2]:

# === CONFIG ===
CSV_PATH = "./data/job_descriptions_large.csv"
CHROMA_DIR = "./chroma_db"


In [3]:

# === STEP 1: Prepare Vector Store ===
if not os.path.exists(CHROMA_DIR):
    print("📦 Creating new Chroma vector DB...")
    df = pd.read_csv(CSV_PATH)

    # Convert rows to LangChain Documents
    documents = [
        Document(page_content=row["Description"], metadata={"title": row["Job Title"]})
        for _, row in df.iterrows()
    ]

    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(documents, embedding=embedding, persist_directory=CHROMA_DIR)
    vector_db.persist()
else:
    print("✅ Loading existing Chroma vector DB...")
    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embedding)


✅ Loading existing Chroma vector DB...


C:\Users\Buddima Dissanayake\AppData\Local\Temp\ipykernel_16560\2771827101.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
f:\FYP latest\chatbot1\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Buddima Dissanayake\AppData\Local\Temp\ipykernel_16560\2771827101.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the 

In [4]:

# === STEP 2: Setup LLM & Prompt ===
llm = Ollama(model="llama3")

prompt_template = PromptTemplate(
    input_variables=["traits", "context"],
    template="""
You are an intelligent and concise career advisor.

A candidate has shared these personality traits:
{traits}

Below are job role descriptions you may use as reference:
{context}

From the roles below, pick the single **best fit**:
- Software Engineer (SE)
- Quality Assurance Engineer (QA)
- Business Analyst (BA)

Your response must include:
1. The best-fit role (SE, QA, or BA)
2. 2-3 traits from the input that support the match
3. A brief explanation in 2–3 sentences

Respond in this exact format:

[Recommended Role]: <SE / QA / BA>
[Supporting Traits]: <trait1>, <trait2>, ...
[Reasoning]: <short explanation>
"""
)


chain = LLMChain(llm=llm, prompt=prompt_template)


C:\Users\Buddima Dissanayake\AppData\Local\Temp\ipykernel_16560\1143978823.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")
C:\Users\Buddima Dissanayake\AppData\Local\Temp\ipykernel_16560\1143978823.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [ ]:

# === STEP 3: Conversational Loop ===
print("\n🧠 Job Recommendation Chatbot (type 'exit' to quit)\n")

while True:
    user_input = input("🧑‍💼 Describe your traits, behaviors, or preferences: ")
    if user_input.lower() == "exit":
        print("👋 Goodbye!")
        break

    # Get similar job docs from vector DB
    docs = vector_db.similarity_search(user_input, k=5)
    context = "\n\n".join([doc.page_content for doc in docs])

    try:
        result = chain.run(traits=user_input, context=context)
        print("\n🤖 Recommendation:\n", result)
        print("-" * 60)
    except Exception as e:
        print("⚠️ Error:", e)



🧠 Job Recommendation Chatbot (type 'exit' to quit)

